In [3]:
import pandas as pd
train_data = pd.read_csv("train.csv")
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
#After check the train.csv, we could know that we missing some 177 age, 687 cabin, 2 Embarked.
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
#Survived < 50%, less than 40% survived.
#Pclass's 25%=2,50%=3, which means p1 <25%,P3>50%.
#Slibsp -> mean 0.5, 75%=1, 50%=0, not too many people know each others.
#parch -> 75% = 0, mean = 0.38, because of the background, we could guess rich people bring children to journel, poor people leave home to work
#fare: mean 32, 75% 31, which means not too many people paid expensive fare ticket.
import numpy as np
train_data.describe(include=[np.object])


,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Parr, Mr. William Henry Marsh",male,1601,C23 C25 C27,S
freq,1,577,7,4,644


In [6]:
#577 male, 314 female.
import matplotlib.pyplot as plt
train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)


,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [7]:
var = train_data.groupby('Sex').Survived.sum()
print(var)

Sex
female    233
male      109
Name: Survived, dtype: int64


In [9]:
X = train_data[['Pclass','Age','Sex']]
y = train_data['Survived']

In [11]:
X['Age'].fillna(X['Age'].mean(), inplace=True)

/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [12]:
#数据分割,拆分训练集和测试集
from sklearn.model_selection import  train_test_split #随机划分测试集合训练集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)


In [13]:
#特征转换
from sklearn.feature_extraction import  DictVectorizer
vec = DictVectorizer(sparse=False)
X_train = vec.fit_transform(X_train.to_dict(orient='record'))
print(vec.feature_names_)

['Age', 'Pclass', 'Sex=female', 'Sex=male']


In [14]:
#对预测数据进行同样的特征转换
X_test = vec.transform(X_test.to_dict(orient='record'))

In [15]:
#导入决策树模型并对测试特征数据进行预测
from sklearn.tree import  DecisionTreeClassifier
#使用默认配置初始化决策树分类器
dtc = DecisionTreeClassifier()
#训练数据进行模型学习
dtc.fit(X_train,y_train)
#决策树模型对特征数据进行预测
y_predict=dtc.predict(X_test)

In [16]:
#模型评估
from sklearn.metrics import  classification_report
#输出预测的准确性
print(dtc.score(X_test,y_test))
#输出详细的分类性能
print(classification_report(y_predict,y_test,target_names=['died','survived']))


0.834080717489
             precision    recall  f1-score   support

       died       0.90      0.84      0.87       143
   survived       0.74      0.82      0.78        80

avg / total       0.84      0.83      0.84       223



In [17]:
#使用随机深林分类器进行生成模型的训练和预测分析
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_y_pred = rfc.predict(X_test)
#输出随机深林分类器在测试集上的分类准确性
print(rfc.score(X_test,y_test))
print(classification_report(rfc_y_pred,y_test,target_names=['died','survived']))

0.829596412556
             precision    recall  f1-score   support

       died       0.87      0.85      0.86       138
   survived       0.76      0.80      0.78        85

avg / total       0.83      0.83      0.83       223



In [18]:
#使用梯度提升决策树进行集成O型的训练和模型预测
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
gbc_y_pred = gbc.predict(X_test)
#输出梯度提升树在测试集上的分类准确性
print(gbc.score(X_test,y_test))
print(classification_report(gbc_y_pred, y_test,target_names=['died','survived']))

0.843049327354
             precision    recall  f1-score   support

       died       0.91      0.84      0.87       145
   survived       0.74      0.85      0.79        78

avg / total       0.85      0.84      0.85       223

